# Raster formats and libraries

This notebook based on content from previous geohackweek raster tutorials https://github.com/geohackweek/raster

<div class="alert-info">

### Overview
    
* **teaching:** 15 minutes
* **exercises:** 0
* **questions:**
    * What file formats are available for representing raster datasets?
* **objectives:**
    * Understand the high-level data interchange formats for raster datasets.
  
</div>

### Table of contents

1. [**Raster Data Format**](#Raster-Data-Format)
    1. [Geotiffs](#About-Raster-Data)
    1. [External metadata](#Metadata)
    1. [Processing levels](#Processing-levels)
1. [**Geospatial Data Abstraction Library (GDAL)**](#Geospatial-Data-Abstraction-Library-(GDAL))
    1. [Converting formats](#Converting-formats)
    1. [Reprojection](#Reprojection)
    1. [Visualization and computation](#Visualization-and-computation)
1. [**Raster Tiles**](#Raster-Tiles)  

### Raster Data Format 

Raster data can come in an astounding variety of [different formats](https://gdal.org/drivers/raster/index.html)! At a minimum, these formats include the following metadata:

1. Geotransform (defines extent, resolution)
2. Coordinate Reference System (CRS)
3. Values that represent missing data (`NoDataValue`) 

### Geotiff

One of the most common raster formats is the `GeoTIFF` which has the extension `.tif`. A GeoTiff consists of a standard `.tif` image file with an embedded header containing metadata and attributes (`tif tags`). 

Your digital camera stores EXIF tags that describe the make and model of the camera, the date the photo was taken, etc. A GeoTIFF can also contain this information, but it also contains additional fields for spatial (georeferencing) information.

Spatially-aware applications such as [QGIS](https://qgis.org) interpret this metadata.  If we aren't careful (or are using a raster-editing application that ignores spatial information), we can accidentally strip this spatial metadata.  Photoshop, for example, can edit GeoTiffs, but we'll lose the embedded CRS and geotransform if we save to the same file!

<div class="alert-warning">

#### More Resources on the  `.tif` format

* [GeoTIFF on Wikipedia](https://en.wikipedia.org/wiki/GeoTIFF)
* [Cloud-optimized Geotiffs (COGs)](https://www.cogeo.org)

</div>

### External metadata

In addition to tags it's very common to have external metadata files that explain many other critical ancilliary details. For example, what software and version was used to generate the image? What time was the image acquired? As with Raster data formats there are also myriad metadata formats. NASA has undertaken an effort to consolidate and standardize metadata across decades of observations from hundreds of sensors. This is an extremely important effort and a complicated undertaking that you can read more about here - [NASA's Common Metadata Repository](https://earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/cmr)

Most scientists and users of raster data need not concern themselves with the details of CMR. Users of raster data need to search, filter, and ensure the quality of satellite images. [SpatioTemporal Asset Catalogs](https://stacspec.org/) are a user-facing sidecar metadata file that is meant to facilitate exactly those needs.


### Processing levels

Geospatial rasters are also oftern referred to by their "processing level". These are usually standard across space agencies, but there can be subtle difference between for example [NASA](https://science.nasa.gov/earth-science/earth-science-data/data-processing-levels-for-eosdis-data-products) and [ESA](https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/processing-levels). `Level 0`, as you might expect, is the lowest-level information collected at the sensor (for example voltage counts). `Level 1` is derived from Level 0 and could consist of an image with non-physical units and minimal metadata. `Level 2` is typically an image where sensor counts are scaled to measure some physical unit (for example, pixel units have a value of temperature). `Level 3` might be re-gridded and reprojected to CRS. `Level 4` is further processed information, for example a timeseries derived from multiple stacked images.

<div class="alert-warning">

#### Important    

As you can imagine, given all the steps and options for processing data, it's not sufficient to say "we'll use the landsat-8 image over Seattle on a specific date". [Which processing level of data are we going to use](https://www.usgs.gov/land-resources/nli/landsat/product-information)? 
    
    
</div>

### Geospatial Data Abstraction Library (GDAL)
[GDAL](http://gdal.org) is the de facto standard library for
interaction and manipulation of geospatial raster data.  The primary purpose of GDAL is to read, write and transform geospatial datasets in a way that makes sense in the context of its spatial metadata.  GDAL also includes a set of [command-line utilities](http://www.gdal.org/gdal_utilities.html) (e.g., `gdalinfo`, `gdal_translate`, `gdalwarp`) for convenient inspection and manipulation of raster data.

GDAL is a powerful and mature library for reading, writing and warping raster datasets, written in C++ with bindings to provide convenient APIs for other languages (e.g., Python, R). 

## GDAL command line utilities

* Learn these: https://www.gdal.org/gdal_utilities.html
    * gdalinfo
    * gdal_translate
    * gdalwarp
    * gdaladdo
* A few tips:
    * Use standard creation options (co)
        * TILED=YES
        * COMPRESS=LZW
        * BIGTIFF=IF_SAFER
    * Choose your resampling algorithm carefully
        * Default is nearest neightbor, fast but can lead to artifacts (best to limit use for categorical raster)
        * Bilinear or bicubic resampling is usually a better choice for continuous rasters

https://medium.com/planet-stories/a-gentle-introduction-to-gdal-part-4-working-with-satellite-data-d3835b5e2971

In [ ]:
# ! runs a terminal command
!gdalinfo --version

GDAL's support for different raster file formats depends on the format drivers that have been implemented, and the libraries that are available at compile time. To find the available formats for your current install of GDAL:

In [ ]:
# Output cropped to 5 lines
!gdalinfo --formats | head -n 5

In [ ]:
# GDAL can operate on local files or even read files stored on a server:
HOST = 'http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1'
IMAGE = 'LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'

In [ ]:
# NOTE we are expanding python variables defined above with {}
!gdalinfo /vsicurl/{HOST}/{IMAGE}

### Converting formats

Often you want files in a specific format. GDAL is great for format conversions. One of the most powerful and useful formats is the `virtual raster` [VRT format](https://www.gdal.org/gdal_vrttut.html). It is essentially an XML file that fully describes a raster, but does not duplicate the binary data. For example, you can save a reference to a remote file to your local disk without downloading the entire file!

In [ ]:
%%bash
# Alternatively you can run a short bash script with %%bash
HOST='http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1'
IMAGE='LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'
gdal_translate -of VRT /vsicurl/$HOST/$IMAGE LC08_L1TP_042034_20170616_20170629_01_T1_B4.vrt

In [ ]:
# Now you can forget about the strange '/vsicurl/' syntax and just work directly with the local file. 
# The command below should give you the same print-out as earlier.
!gdalinfo LC08_L1TP_042034_20170616_20170629_01_T1_B4.vrt | grep PROJCS

### Reprojection

Another common task is warping an image to a different coordinate system. Note from above that the file we are working with has a CRS of UTM 11N. [Universal Transverse Mercator](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) is a very common raster format for small bounding boxes because the x and y dimensions are in units of meters. 

Whenever you want to convert from one CRS to another, it is extremely useful to use [EPSG Codes](https://spatialreference.org/ref/epsg/). These codes are essentially a  shorthand and internationally agreed upon database of all the CRS's out there. Two of the most common ones that are worth memorizing are `4326` which is unprojected WGS84 Lat/Lon, and `3857` which is Google Mercator, used extensively for maps on the web. 

The example command below warps the image from UTM Coordinates to WGS84 lat/lon coordinates:

In [ ]:
!gdalwarp -t_srs EPSG:4326 -of VRT /vsicurl/{HOST}/{IMAGE} LC08_L1TP_042034_20170616_20170629_01_T1_B4-wgs84.vrt

In [ ]:
# Confirm reprojection by looking at the new coordinates
# Note the pixel size and corner coordinates are now in units of degrees
!gdalinfo LC08_L1TP_042034_20170616_20170629_01_T1_B4-wgs84.vrt

## Visualization and computation

This just scratches the surface of what GDAL is capable of. Next you might want to look into sampling raster values with [`gdallocationinfo`](https://gdal.org/programs/gdal_calc.html) or performing computations with [`gdal_calc`](https://gdal.org/programs/gdal_calc.html). GDAL itself does not handle graphics for visualizing raster data, but it underpins many graphical GIS programs, such as [QGIS](https://qgis.org). We'll look at visualization with Python libraries in subsequent notebooks.

## Raster Tiles

When rasters cover very large areas, or are very high resolution, it is convenient to chop them up into discrete tiles that can be reassembled into the full image. This is especially true for interactive web maps where we want to visualize a raster at a resolution appropriate for a given zoom level. If you are looking at the entire globe, there is no reason to render an image at 30m resolution because our eyes can't discern the detail. Conversly if we are zoomed into a city, we want the full resolution data displayed. This schematic illustrates tiling:

<img src="../assets/ArcGIS-raster-tiles.png" width="300"/>
*Source: http://desktop.arcgis.com.*

It is important to note that the bounding boxes of tiles, as well as the pixel averaging scheme affects how the raster appears at a given zoom level. There are common standard tiling schemes for web maps - check out the [mercantile library](https://github.com/mapbox/mercantile). GDAL also offers utilities to easily generate tiles from any raster. For example, see [`gdal2tiles.py`](https://gdal.org/programs/gdal2tiles.html)  or the [`MBTiles` format](https://gdal.org/drivers/raster/mbtiles.html).


<div class="alert-info">

## key points 

- There are myriad formats for storing geospatial metadata and accompanying metadata (for example Geotiff)
- Cloud-optimized Geotiff (COG) and Spatio-Temporal Asset Catalogs (STAC) are important modern formats
- The Geospatial Data Abstraction Library (GDAL) is very useful for reading, writing and transforming rasters 
- GDAL is not a visualization tool, but underpins full-featured GIS software like QGIS
- Tiled rasters are increasingly common for high-resolution rasters and web-based visualization
    
</div>